# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [73]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

## Instantiate a Spark session

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data Wrangling with DataFrames Coding Quiz") \
    .getOrCreate()

22/09/19 04:27:09 WARN Utils: Your hostname, Lucianos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.191 instead (on interface en0)
22/09/19 04:27:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/19 04:27:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/19 04:27:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/19 04:27:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/19 04:27:12 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


## Read in the data set located at the path "data/sparkify_log_small.json"

In [3]:
df = spark.read.json("data/sparkify_log_small.json")

# Explore

In [7]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
df.select("artist").show()

+--------------------+
|              artist|
+--------------------+
|       Showaddywaddy|
|          Lily Allen|
|Cobra Starship Fe...|
|          Alex Smoke|
|                null|
|                null|
|              Redman|
|     Ulrich Schnauss|
|                null|
|                null|
|               Jay-Z|
|         Evanescence|
|     Scissor Sisters|
|        3 Doors Down|
|       George Younce|
|              Aly-Us|
|                null|
|            BjÃÂ¶rk|
|      David Bromberg|
|          Nickelback|
+--------------------+
only showing top 20 rows



In [10]:
df.select("artist").dropDuplicates().show()

+--------------------+
|              artist|
+--------------------+
|      The Black Keys|
|        STRATOVARIUS|
|      The Chameleons|
|Dashboard Confess...|
|      Jarabe De Palo|
|        Ziggy Marley|
|        Yann Tiersen|
|  The Watts Prophets|
|            Goldfish|
|           Kate Nash|
|              DJ Taz|
|    Jane's Addiction|
|         Eva Cassidy|
|               Rufio|
|           Los Lobos|
|         Silverstein|
|        Rhett Miller|
|              Nebula|
|Yonder Mountain S...|
|     Groove Coverage|
+--------------------+
only showing top 20 rows



In [12]:
df.select("artist").dropDuplicates().count()

3618

In [22]:
df.createOrReplaceTempView("EMP")

In [26]:
spark.sql("select artist, count(*) as count from EMP GROUP BY artist ORDER BY count desc").show(truncate=False)

+----------------------------------------------------------------------+-----+
|artist                                                                |count|
+----------------------------------------------------------------------+-----+
|null                                                                  |1653 |
|Coldplay                                                              |83   |
|Kings Of Leon                                                         |69   |
|Florence + The Machine                                                |52   |
|BjÃÂ¶rk                                                              |46   |
|Dwight Yoakam                                                         |45   |
|Justin Bieber                                                         |43   |
|The Black Keys                                                        |40   |
|OneRepublic                                                           |37   |
|Jack Johnson                                       

# Question 1

Which page did user id "" (empty string) NOT visit?

In [27]:
spark.sql("select distinct page from EMP WHERE userId = ''").show(truncate=False)

+-----+
|page |
+-----+
|Home |
|About|
|Login|
|Help |
+-----+



In [37]:
df.select("page").filter(df.userId=="").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?

Users who are only visiting the About, Home and Login pages are probably unregistered visitors.

# Question 3

How many female users do we have in the data set?

In [45]:
spark.sql("select gender, count(distinct(userId)) as count from EMP WHERE gender = 'F' GROUP BY gender").show(truncate=False)

+------+-----+
|gender|count|
+------+-----+
|F     |462  |
+------+-----+



In [58]:
df.select("userId").distinct().filter(df.gender=="F").count()

462

# Question 4

How many songs were played from the most played artist?

In [77]:
spark.sql("select artist, count(*) as count from EMP WHERE artist IS NOT NULL GROUP BY artist ORDER BY count desc LIMIT 1").show(truncate=False)

+--------+-----+
|artist  |count|
+--------+-----+
|Coldplay|83   |
+--------+-----+



In [79]:
df.select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [80]:
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

